# Import Library

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
import datetime as dt
import pandas as pd
import os
import xml.etree.ElementTree as ET

In [3]:
%cd /gdrive/MyDrive/Colab_Weather_API/lib
import new_weather as nw
import weather_dict as wd
from importlib import reload

/gdrive/MyDrive/Colab_Weather_API/lib


In [4]:
%cd /gdrive/MyDrive/Colab_Weather_API/lib
reload(nw)

<module 'new_weather' from '/gdrive/MyDrive/Colab_Weather_API/lib/new_weather.py'>

In [109]:
%cd /gdrive/MyDrive/Colab_Weather_API/lib
reload(wd)

/gdrive/MyDrive/Colab_Weather_API/lib


<module 'weather_dict' from '/gdrive/MyDrive/Colab_Weather_API/lib/weather_dict.py'>

# LOG

In [6]:
tz = dt.timezone(dt.timedelta(hours=9))
dt_now = dt.datetime.now(tz=tz)
str_now = dt.datetime.strftime(dt_now, '%Y-%m-%d %H:%M:%S')
# print(f"# 작성시간:\t{str_now}")
# print("# 작성자:\t\tjongphago")
# print("# 주제:\t\t함수'change_args' Error")

In [7]:
# 작성시간: 2021-06-28 17:14:19
# 작성자:   jongphago
# 주제:     함수'change_args' Error
"""
문제점:
    - arg_dict를 생성할때 YYYYmmdd 형식으로 저장하여 시간 정보를 무시한다.
    - 함수 change_args를 호출할때 변경해야 하는 시간 값은 정확히 999hrs후 
    값인데, 시간정보가 없어서 특정 시간만큼 추가되는 현상이 발생한다.
    
해결책:
    Alt1:
        - makre_arg_dict 함수를 작성할때 날짜관련 key값이 '년-월-일-시-분' 
        데이터를 모두 갖도록 작성한다
        - 장점: 정확한 시간(ex:999시간 후) 단위로 컨트롤이 가능
        - 단점: 많은 함수를 수정해야 한다. 미리 고생.
    Alt2:
        - 일단위로 K시간을 입력하면 %24 만큼의 시간만 입력 되도록 한다.
        - 장점: 수정할 함수가 적음
        - 단점: 정확한 컨트롤이 어려움. 앞으로 고생.
""" 
pass

# serviceKey 생성

In [8]:
lib_path = "/gdrive/MyDrive/Colab_PRIVATE/weather_api"
service_key = nw.get_service_key(lib_path)

# Info Dictionary

In [10]:
call_info = pd.DataFrame(wd.call_dict).transpose()
call_info

,항목명,항목크기,항목구분,샘플데이터,항목설명
serviceKey,인증키,100,1,인증키,공공데이터포털에서 발급받은 인증키(URL_Encode)
numOfRows,한_페이지_결과_수,4,0,10,한 페이지 결과 수(Default: 10)
pageNo,페이지_번호,4,0,1,페이지 번호(Default: 1)
dataType,응답자료형식,4,0,XML,요청자료형식(XML/JSON)(Default: XML)
dataCd,자료_코드,4,1,ASOS,자료 분류 코드
dateCd,날짜_코드,3,1,HR,날짜 분류 코드
startDt,시작일,8,1,20100101,조회 기간 시작일
startHh,시작시,2,1,1,조회 기간 시작시
endDt,종료일,8,1,20100601,조회 기간 종료일((전일(D-1) 까지 제공))
endHh,종료시,2,1,1,조회 기간 종료시


##Error dict

In [11]:
error_info = pd.DataFrame(wd.error_dict).transpose()
error_info

,msg,discription
0,NORMAL_SERVICE,정상
1,APPLICATION_ERROR,어플리케이션 에러
2,DB_ERROR,데이터베이스 에러
3,NODATA_ERROR,데이터없음 에러
4,HTTP_ERROR,HTTP 에러
5,SERVICETIME_OUT,서비스 연결실패 에러
10,INVALID_REQUEST_PARAMETER_ERROR,잘못된 요청 파라메터 에러
11,NO_MANDATORY_REQUEST_PARAMETERS_ERROR,필수요청 파라메터가 없음
12,NO_OPENAPI_SERVICE_ERROR,해당 오픈API서비스가 없거나 폐기됨
20,SERVICE_ACCESS_DENIED_ERROR,서비스 접근거부


##컬럼명 해석기

In [12]:
asos_columns_info = pd.DataFrame(wd.asos_dict).transpose()
asos_columns_info.head()

,항목명,항목크기,샘플데이터,설명
numOfRows,한 페이지 결과 수,4,1,한 페이지당 표출 데이터 수
pageNo,페이지 번호,4,1,페이지 수
totalCount,데이터 총 개수,10,1,데이터 총 개수
resultCode,응답메시지 코드,2,0,응답 메시지코드
resultMsg,응답메시지 내용,100,NORMAL SERVICE,응답 메시지 설명


In [13]:
asos_columns_info[['항목명', '샘플데이터']].transpose()

,numOfRows,pageNo,totalCount,resultCode,resultMsg,dataType,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,wsQcflg,wd,wdQcflg,hm,hmQcflg,pv,td,pa,paQcflg,ps,psQcflg,ss,ssQcflg,icsr,dsnw,hr3Fhsc,dc10Tca,dc10LmcsCa,clfmAbbrCd,lcsCh,vs,gndSttCd,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te
항목명,한 페이지 결과 수,페이지 번호,데이터 총 개수,응답메시지 코드,응답메시지 내용,데이터 타입,시간,목록 순서,지점 번호,서울,기온,기온 품질검사,강수량,강수량 품질검사,풍속,풍속 품질검사,풍향,풍향 품질검사,습도,습도 품질검사,증기압,이슬점온도,현지기압,현지기압 품질검사,해면기압,해면기압 품질검사,일조,일조 품질검사,일사,적설,3시간신적설,전운량,중하층운량,운형,최저운고,시정,지면상태,현상번호,지면온도,지면온도 품질검사,5cm 지중온도,10cm 지중온도,20cm 지중온도,30cm 지중온도
샘플데이터,1,1,1,0,NORMAL SERVICE,XML,2010-01-01 10,1,108,서울,23.8,0,10.5,0,1,0,110,0,36,0,1.1,-21.4,1012.4,0,1023.6,0,1,0,0.73,2.2,0.2,0,0,scas,8,2300,17,1904,-3.4,0,-4.9,-2.4,-1,0.4


In [14]:
wd.translate(wd.asos_dict, 'ts')

'지면온도'

## 관측소 정보

In [15]:
station_info = pd.DataFrame(wd.station_dict).transpose()
station_info

,stnNm,manager
90,속초,강원지방기상청
93,북춘천,춘천기상대
95,철원,강원지방기상청
98,동두천,수도권기상청
99,파주,수도권기상청
...,...,...
285,합천,울산기상대
288,밀양,울산기상대
289,산청,창원기상대
294,거제,부산지방기상청


In [16]:
station_info.index

Int64Index([ 90,  93,  95,  98,  99, 100, 101, 102, 104, 105, 106, 108, 112,
            114, 115, 119, 121, 127, 129, 130, 131, 133, 135, 136, 137, 138,
            140, 143, 146, 152, 155, 156, 159, 162, 165, 168, 169, 170, 172,
            174, 177, 184, 185, 188, 189, 192, 201, 202, 203, 211, 212, 216,
            217, 221, 226, 232, 235, 236, 238, 239, 243, 244, 245, 247, 248,
            251, 252, 253, 254, 255, 257, 258, 259, 260, 261, 262, 263, 264,
            266, 268, 271, 272, 273, 276, 277, 278, 279, 281, 283, 284, 285,
            288, 289, 294, 295],
           dtype='int64')

# weather_args 생성

In [17]:
arg_dict = nw.make_arg_dict(112, '20210101', '20210601')
arg_dict['_numOfRows']

3648

# Call API

In [18]:
master_df = nw.call(service_key, arg_dict)

In [19]:
# master_df.shape
master_df.head()

,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,wsQcflg,wd,wdQcflg,hm,hmQcflg,pv,td,pa,paQcflg,ps,psQcflg,ss,ssQcflg,icsr,dsnw,hr3Fhsc,dc10Tca,dc10LmcsCa,clfmAbbrCd,lcsCh,vs,gndSttCd,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te
0,2021-01-01 00:00,1,112,인천,-7.5,None,None,9,2.0,None,20,None,62,None,2.2,-13.5,1018.4,None,1027.4,None,None,9,None,None,None,0,0,None,None,1657,None,None,-8.7,None,-1.2,0.2,1.3,2.3
1,2021-01-01 01:00,2,112,인천,-7.8,None,None,None,1.9,None,50,None,63,None,2.1,-13.6,1018.3,None,1027.4,None,None,9,None,None,None,0,0,None,None,1852,None,None,-8.7,None,-1.2,0.2,1.3,2.4
2,2021-01-01 02:00,3,112,인천,-7.8,None,None,None,1.0,None,20,None,61,None,2.1,-14.0,1018.3,None,1027.4,None,None,9,None,None,None,2,0,None,None,1745,None,None,-9.0,None,-1.3,0.1,1.3,2.2
3,2021-01-01 03:00,4,112,인천,-8.0,None,None,None,1.9,None,50,None,60,None,2.0,-14.4,1018.6,None,1027.7,None,None,9,None,None,None,0,0,None,None,1950,None,None,-9.0,None,-1.4,0.1,1.2,2.2
4,2021-01-01 04:00,5,112,인천,-7.9,None,None,None,1.9,None,70,None,58,None,2.0,-14.7,1017.8,None,1026.9,None,None,9,None,None,None,0,0,None,None,1937,None,None,-9.0,None,-1.5,0.0,1.2,2.2


## Flag 보유 컬럼

In [20]:
flag_columns_list = []
for column in master_df.columns:
    if column.endswith('Qcflg'):
        flag_columns_list.append(column)

In [21]:
flag_list =[columns[:2] for columns in flag_columns_list]

In [22]:
# master_df[flag_list]

# 운형

In [25]:
wd.cloud_name(3)

'Ac'

In [26]:
wd.cloud_label('Ac')

3

In [58]:
pd.DataFrame(master_df['clfmAbbrCd'].value_counts()).transpose()

,Ci,Sc,ScAs,StNs,ScCi,St,As,Ac,AcCi,ScAc,CbStNs,Cs,CsCi,ScCs,AcCs,ScAcCi,AsCi,ScCsCi,ScAsCi
clfmAbbrCd,581,394,203,174,113,92,73,64,62,28,8,8,7,6,3,2,1,1,1


## Multi label one hot encoding

In [107]:
master_df.drop(columns='clfmIdx', inplace=True)

In [111]:
# cloud form index
if not any(master_df.columns.isin(['clfmIdx'])):
    print("컬럼 'clfmIdx'을 추가합니다.")
    master_df['clfmIdx'] = master_df['clfmAbbrCd'].apply(wd.function)
else:
    print("컬럼 'clfmIdx'이 존재합니다.")

컬럼 'clfmIdx'이 존재합니다.


In [121]:
master_df['clfmIdx'][~(master_df['clfmAbbrCd'].isna())]

5         8.0
6         8.0
7         8.0
8         8.0
9         8.0
        ...  
3639    520.0
3640     40.0
3641    576.0
3642    512.0
3645    512.0
Name: clfmIdx, Length: 1821, dtype: float64

# 결측치 처리

##결측치 확인

In [ ]:
pd.DataFrame(master_df.isnull().sum()/master_df.shape[0], columns=['null_ratio'])

In [ ]:
wd.error_dict[02]

In [ ]:
wd.flag_dict

In [ ]:
master_df.columns

In [ ]:
display(master_df['rn'].iloc[:60] == '0.0')

In [ ]:
# pd.options.display.max_rows

In [ ]:
# '결측'
master_df['rnQcflg'].value_counts()

In [ ]:
# '정상'
master_df['rnQcflg'].isna().sum()